In [1]:
from nltk.corpus import movie_reviews

In [2]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

In [7]:
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]

In [9]:
texts = [" ".join(f) for f in negfeats]  + [" ".join(f) for f in posfeats]
labels = [0] * len(negfeats) + [1] * len(posfeats)

In [10]:
texts[0]

'plot : two teen couples go to a church party , drink and then drive . they get into an accident . one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . what \' s the deal ? watch the movie and " sorta " find out . . . critique : a mind - fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . which is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn \' t snag this one correctly . they seem to have taken this pretty neat concept , but executed it terribly . so what are the problems with the movie ? well , its main problem is that it \' s simply too jumbled . it starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience member , have no 

In [16]:
labels[990:1010]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [17]:
len(texts) 

2000

In [19]:
proportion1 = sum(labels)/len(labels)
proportion1

0.5

Импортируйте CountVectorizer из sklearn.feature_extraction.text. Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод fit_transform у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [23]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

In [24]:
X

<2000x39659 sparse matrix of type '<class 'numpy.int64'>'
	with 666842 stored elements in Compressed Sparse Row format>

Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy.

In [25]:
pipeline = Pipeline(
            [('vectorizer', CountVectorizer()),
             ('classifier', LogisticRegression())]
            )

In [26]:
mean_accuracy = cross_val_score(pipeline,texts,labels).mean()
mean_accuracy

C:\Users\artem\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\artem\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8360216503929078

Аналогично accuracy, оцените качество по ROC AUC.

In [28]:
mean_AUC_ROC = cross_val_score(pipeline,texts,labels,scoring = 'roc_auc').mean()
mean_AUC_ROC

C:\Users\artem\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\artem\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9107764937833774

Обучите логистическую регрессию на всей доступной вам выборке и выведите 5 наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer.

In [29]:
classifier = LogisticRegression()

In [30]:
classifier.fit(X,labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [64]:
import numpy as np
features = np.array(vectorizer.get_feature_names())
features

array(['00', '000', '0009f', ..., 'zwigoff', 'zycie', 'zzzzzzz'],
      dtype='<U58')

In [36]:
#vectorizer.vocabulary_

In [62]:
weights = classifier.coef_[0]
weights

array([ 1.13520099e-02, -1.78936849e-02,  2.51623696e-06, ...,
       -7.15498559e-03,  3.79019244e-04, -1.40853471e-03])

In [74]:
type(weights[0])

numpy.float64

In [70]:
import pandas as pd
df = pd.DataFrame(np.array([features,np.abs(weights)]).T)
df


,0,1
0,00,0.011352009883429385
1,000,0.017893684911230296
2,0009f,2.5162369572254385e-06
3,007,0.024638828832084736
4,00s,0.0003407404584646016
5,03,0.0057174972662168725
6,04,0.0028587486331084363
7,05,0.01888753664882236
8,05425,0.0028587486331084363
9,10,0.16853634697258033


In [92]:
df[2] = df[1].apply(lambda x: float(x))

In [93]:
df.head()

,0,1,2
0,00,0.011352009883429385,0.011352
1,000,0.017893684911230296,0.017894
2,0009f,2.5162369572254385e-06,0.000003
3,007,0.024638828832084736,0.024639
4,00s,0.0003407404584646016,0.000341


In [100]:
df.sort_values(by=[2],ascending=False, kind='mergesort')

,0,1,2
2954,bad,0.7821764028113708,7.821764e-01
37056,unfortunately,0.6366187230277182,6.366187e-01
39195,worst,0.5929017021948101,5.929017e-01
14159,fun,0.5560662734737746,5.560663e-01
38417,waste,0.5081787289571051,5.081787e-01
24063,nothing,0.5039888510322149,5.039889e-01
30874,script,0.46653419940839563,4.665342e-01
2847,awful,0.4652169287985426,4.652169e-01
4425,boring,0.4631911361466869,4.631911e-01
24521,only,0.45995729312103356,4.599573e-01


In [ ]:
bad
unfortunately
worst
fun
waste
